In [1]:
# !pip install --upgrade azure-cognitiveservices-vision-computervision

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
import const
import json

subscription_key = const.AZURE_KEY
endpoint = "https://textvqa.cognitiveservices.azure.com/"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [2]:
print("===== Batch Read File - remote =====")
# Get an image with handwritten text
# remote_image_handw_text_url = "https://raw.githubusercontent.com/MicrosoftDocs/azure-docs/master/articles/cognitive-services/Computer-vision/Images/readsample.jpg"

# Call API with URL and raw response (allows you to get the operation location)
# recognize_handw_results = computervision_client.read(remote_image_handw_text_url,  raw=True)

def word2dic(w):
    ret = {}
    for att in w.__dict__.keys():
        ret[att] = w.__dict__[att]
    return ret

def line2dic(l):
    ret = {}
    for att in l.__dict__.keys():
        if att!="words":
            ret[att] = l.__dict__[att]
        else:
            ret["words"] = []
            for w in l.words:
                ret["words"].append(word2dic(w))
    return ret

def res2dic(res):
    ret = {}
    for att in res.__dict__.keys():
        if att!="lines":
            ret[att] = res.__dict__[att]
        else:
            ret["lines"] = []
            for l in res.lines:
                ret["lines"].append(line2dic(l))
    return ret  

===== Batch Read File - remote =====


In [ ]:
for _,_,ls in os.walk(const.IMG_PATH):
    pred_ls = ls

error_f = open("./azocr_fail_imgs_retry.txt","w")
err_cnt = 0
OFFSET = 10

for cnt, img in enumerate(pred_ls[OFFSET:]):
    if img.endswith(".json"):
        continue
    print(cnt+OFFSET,len(pred_ls),err_cnt,img,end="\r")
    pred_im_path = os.path.join(const.IMG_PATH,img)
    try:
        recognize_handw_results = computervision_client.read_in_stream(open(pred_im_path, 'rb'),raw=True)
    except:
        error_f.write(f"{cnt}\t{img}\tIO-ERROR\n")
        err_cnt+=1
        continue
    # Get the operation location (URL with an ID at the end) from the response
    operation_location_remote = recognize_handw_results.headers["Operation-Location"]
    # Grab the ID from the URL
    operation_id = operation_location_remote.split("/")[-1]

    # Call the "GET" API and wait for it to retrieve the results 
    while True:
        get_handw_text_results = computervision_client.get_read_result(operation_id)
        if get_handw_text_results.status not in ['notStarted', 'running']:
            break
        time.sleep(1)
        #print("SLEPT!")

    # Print the detected text, line by line
    if get_handw_text_results.status == OperationStatusCodes.succeeded:
        out_f = open(os.path.join(const.AZURE_OCR_PATH,img.split(".")[0]+".json"),"w")
        results = get_handw_text_results.analyze_result.read_results
        if len(results)>1:
            raise NotImplementedError
        json.dump(res2dic(get_handw_text_results.analyze_result.read_results[0]),out_f)
        out_f.close()
        #print(outstr)
        #print("SUCCEED!", img)
    else:
        #print("WRONG!", img)
        error_f.write(f"{cnt}\t{img}\tOP-ERROR\n")
        err_cnt+=1

print()
print("DA CHENG GONG!")
error_f.close()